# 🌍 MASt3R-SLAM + Stella World Builder

This notebook processes a video using MASt3R-SLAM to create:
1. **3D Point Cloud** (PLY file)
2. **.stella Explorable World** (ZIP container with collision and render mesh)

## Instructions:
1. Click **Runtime → Change runtime type** and select **GPU** (T4 is fine)
2. Run cells one by one (Shift+Enter)
3. Upload your video when prompted
4. Download the outputs at the end

**Estimated time:** 30-45 minutes total

## Step 1: Check GPU

In [ ]:
# Check GPU is available
!nvidia-smi
import torch
if torch.cuda.is_available():
    print(f"\n✅ GPU detected: {torch.cuda.get_device_name(0)}")
else:
    raise RuntimeError("❌ No GPU! Go to Runtime → Change runtime type → GPU")

## Step 2: Clone MASt3R-SLAM

In [ ]:
# Clean start - remove any existing installation
!rm -rf /content/MASt3R-SLAM
!rm -rf /content/lietorch

# Clone MASt3R-SLAM
!git clone --recursive https://github.com/rmurai0610/MASt3R-SLAM.git /content/MASt3R-SLAM
%cd /content/MASt3R-SLAM

# Verify we're in the right place
!pwd
!ls -la

## Step 3: Install Dependencies (10-15 minutes)

In [ ]:
%%time
# lietorch is notoriously hard to build. Let's try multiple approaches.

print("🔧 Step 1: Setup environment...")
!pip uninstall -y lietorch 2>/dev/null || true

# Check what CUDA/PyTorch we have
import torch
print(f"   PyTorch: {torch.__version__}")
print(f"   CUDA: {torch.version.cuda}")

print("\n🔧 Step 2: Try pre-built lietorch wheels...")
# Try the official lietorch PyPI package first
!pip install lietorch 2>&1 | tail -5

# Check if it works
lietorch_ok = False
try:
    # Need to restart Python to pick up new lietorch
    import importlib
    import lietorch
    importlib.reload(lietorch)
    lietorch_ok = True
    print("   ✅ lietorch from PyPI works!")
except Exception as e:
    print(f"   ❌ PyPI lietorch failed: {e}")

if not lietorch_ok:
    print("\n🔧 Step 2b: Building lietorch from source with verbose output...")
    !rm -rf /content/lietorch
    !git clone https://github.com/princeton-vl/lietorch.git /content/lietorch
    
    # Show the actual build error
    print("Building (this will show errors if any)...")
    !cd /content/lietorch && pip install . 2>&1 | tail -50
    
    # Try importing
    try:
        import sys
        if '/content/lietorch' not in sys.path:
            sys.path.insert(0, '/content/lietorch')
        import lietorch
        lietorch_ok = True
        print("   ✅ lietorch built from source!")
    except Exception as e:
        print(f"   ❌ Source build failed: {e}")

if not lietorch_ok:
    print("\n⚠️ LIETORCH FAILED TO INSTALL")
    print("This is a known issue with lietorch + Colab.")
    print("\nAlternative: Use a cloud service with better CUDA support,")
    print("or try RunPod/Lambda Labs which have lietorch pre-configured.")
    raise RuntimeError("Cannot proceed without lietorch")

print("\n🔧 Step 3: Installing other dependencies...")
!pip install -q roma pyglet einops trimesh scipy plyfile
!pip install -q opencv-python-headless

print("🔧 Step 4: Installing curope...")
!cd /content/MASt3R-SLAM/thirdparty/mast3r/dust3r/croco/models/curope && pip install -q . 2>&1 | tail -3

print("🔧 Step 5: Installing asmk + MASt3R...")
!pip install -q -e /content/MASt3R-SLAM/thirdparty/mast3r/asmk 2>&1 | tail -1
!pip install -q --no-deps -e /content/MASt3R-SLAM/thirdparty/mast3r 2>&1 | tail -1

print("🔧 Step 6: Installing in3d...")
!pip install -q glfw pyglm moderngl==5.12.0 moderngl-window==2.4.6 2>&1 | tail -1
!pip install -q --no-deps -e /content/MASt3R-SLAM/thirdparty/in3d 2>&1 | tail -1

print("�� Step 7: Installing MASt3R-SLAM...")
!pip install -q --no-deps -e /content/MASt3R-SLAM 2>&1 | tail -1

print("\n" + "="*60)
print("VERIFICATION")
print("="*60)
!python -c "import torch; print(f'PyTorch {torch.__version__}')"
!python -c "import lietorch; print('lietorch: OK')"
!python -c "import mast3r; print('mast3r: OK')"
print("="*60)
print("\n✅ Ready to process video!")

## Step 4: Download Model Checkpoints (~1.5GB)

In [ ]:
%cd /content/MASt3R-SLAM
!mkdir -p checkpoints

print("Downloading checkpoints (~1.5GB total)...")
!wget -q --show-progress -P checkpoints/ https://download.europe.naverlabs.com/ComputerVision/MASt3R/MASt3R_ViTLarge_BaseDecoder_512_catmlpdpt_metric.pth
!wget -q --show-progress -P checkpoints/ https://download.europe.naverlabs.com/ComputerVision/MASt3R/MASt3R_ViTLarge_BaseDecoder_512_catmlpdpt_metric_retrieval_trainingfree.pth
!wget -q --show-progress -P checkpoints/ https://download.europe.naverlabs.com/ComputerVision/MASt3R/MASt3R_ViTLarge_BaseDecoder_512_catmlpdpt_metric_retrieval_codebook.pkl

print("\n✅ Checkpoints downloaded!")
!ls -lh checkpoints/

## Step 5: Upload Your Video

In [ ]:
from google.colab import files
import os

%cd /content/MASt3R-SLAM

print("📹 Upload your video file (MP4, MOV, AVI)")
print("Click 'Choose Files' below...\n")

uploaded = files.upload()

video_filename = list(uploaded.keys())[0]
video_name = os.path.splitext(video_filename)[0]

print(f"\n✅ Video uploaded: {video_filename}")
print(f"   Size: {os.path.getsize(video_filename) / (1024*1024):.1f} MB")

## Step 6: Run MASt3R-SLAM (15-30 minutes)

In [ ]:
%%time
import os

# Make sure we're in the right directory
%cd /content/MASt3R-SLAM

# Final verification
print("Pre-flight check...")
!python -c "import lietorch; print('✅ lietorch OK')"

print(f"\n🚀 Running MASt3R-SLAM on {video_filename}...")
print("This takes 15-30 minutes. Don't close this tab!\n")

# Run with verbose output
!python main.py \
    --dataset "{video_filename}" \
    --save-as "{video_name}" \
    --config config/base.yaml \
    --no-viz

# Check output
ply_path = f"logs/{video_name}/{video_name}.ply"
if os.path.exists(ply_path):
    size = os.path.getsize(ply_path) / (1024*1024)
    print(f"\n✅ SUCCESS! Point cloud created: {ply_path} ({size:.1f} MB)")
else:
    print(f"\n❌ FAILED - No output file at {ply_path}")
    print("Check the error messages above.")

## Step 7: Create .stella World File

In [ ]:
import numpy as np
import trimesh
import zipfile
import json
import struct
from pathlib import Path
import tempfile

def load_ply(ply_path):
    mesh = trimesh.load(ply_path)
    points = np.array(mesh.vertices)
    colors = None
    if hasattr(mesh, 'visual') and hasattr(mesh.visual, 'vertex_colors'):
        colors = np.array(mesh.visual.vertex_colors)[:, :3]
    return points, colors

def voxelize(points, voxel_size=0.1):
    min_b = points.min(axis=0)
    max_b = points.max(axis=0)
    dims = np.ceil((max_b - min_b) / voxel_size).astype(int) + 1
    coords = np.floor((points - min_b) / voxel_size).astype(int)
    coords = np.clip(coords, 0, dims - 1)
    grid = np.zeros(dims, dtype=bool)
    grid[coords[:, 0], coords[:, 1], coords[:, 2]] = True
    return grid, min_b, voxel_size

def write_rlevox(path, grid, voxel_size, origin):
    with open(path, 'wb') as f:
        f.write(b'STVX')
        f.write(struct.pack('<I', 1))
        f.write(struct.pack('<III', *grid.shape))
        f.write(struct.pack('<f', voxel_size))
        f.write(struct.pack('<fff', *origin))
        flat = grid.astype(np.uint8).flatten()
        rle = []
        i = 0
        while i < len(flat):
            val = flat[i]
            count = 1
            while i + count < len(flat) and flat[i + count] == val and count < 255:
                count += 1
            rle.append(bytes([val, count]))
            i += count
        payload = b''.join(rle)
        f.write(struct.pack('<I', len(payload)))
        f.write(b'\x00' * 28)
        f.write(payload)

def create_stella(ply_path, output_path, title):
    print(f"Loading: {ply_path}")
    points, colors = load_ply(ply_path)
    print(f"Points: {len(points)}")
    
    # Align floor
    floor_y = np.percentile(points[:, 1], 5)
    points[:, 1] -= floor_y
    
    # Voxelize
    print("Voxelizing...")
    grid, origin, vs = voxelize(points, 0.1)
    print(f"Grid: {grid.shape}, {grid.sum()} voxels")
    
    # Create mesh from points
    print("Creating mesh...")
    sphere = trimesh.creation.icosphere(subdivisions=0, radius=0.02)
    meshes = []
    step = max(1, len(points) // 5000)
    for i in range(0, min(len(points), 5000), 1):
        idx = i * step if i * step < len(points) else i
        s = sphere.copy()
        s.apply_translation(points[idx])
        if colors is not None and idx < len(colors):
            c = colors[idx]
            if c.max() <= 1:
                c = (c * 255).astype(np.uint8)
            s.visual.vertex_colors = np.tile(np.append(c, 255), (len(s.vertices), 1))
        meshes.append(s)
    mesh = trimesh.util.concatenate(meshes)
    
    # Create package
    manifest = {
        "schema": "https://virgil.systems/schemas/stella/manifest/v1.schema.json",
        "name": title,
        "version": "1.0.0",
        "levels": [{"id": "0", "name": "Main"}]
    }
    level = {
        "schema": "https://virgil.systems/schemas/stella/level/v1.schema.json",
        "name": "Main Level",
        "spawn": {"position": [float(origin[0]), 1.7, float(origin[2])], "yaw_degrees": 0.0},
        "render": {"uri": "render.glb"},
        "collision": {"uri": "collision.rlevox", "player": {"height_m": 1.7, "radius_m": 0.3}}
    }
    
    with tempfile.TemporaryDirectory() as tmp:
        tmp = Path(tmp)
        write_rlevox(tmp / "collision.rlevox", grid, vs, origin)
        mesh.export(str(tmp / "render.glb"))
        
        with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zf:
            zf.writestr('manifest.json', json.dumps(manifest, indent=2))
            zf.writestr('levels/0/level.json', json.dumps(level, indent=2))
            zf.write(tmp / "render.glb", 'levels/0/render.glb')
            zf.write(tmp / "collision.rlevox", 'levels/0/collision.rlevox')
    
    print(f"✅ Created: {output_path}")

# Run
ply_file = f"logs/{video_name}/{video_name}.ply"
stella_file = f"/content/{video_name}.stella"

if os.path.exists(ply_file):
    create_stella(ply_file, stella_file, title=video_name.replace('_', ' ').title())
else:
    print(f"❌ PLY file not found: {ply_file}")
    print("MASt3R-SLAM may have failed. Check Step 6 output.")

## Step 8: Download Results

In [ ]:
from google.colab import files
import os

print("📥 Downloading files...\n")

# Download PLY
ply_path = f"/content/MASt3R-SLAM/logs/{video_name}/{video_name}.ply"
if os.path.exists(ply_path):
    print(f"Downloading: {video_name}.ply")
    files.download(ply_path)

# Download .stella
stella_path = f"/content/{video_name}.stella"
if os.path.exists(stella_path):
    print(f"Downloading: {video_name}.stella")
    files.download(stella_path)

print("\n✅ Done! Check your Downloads folder.")

## 🎉 Done!

You now have:
- **`.ply`** - 3D point cloud (open in MeshLab, CloudCompare, or Blender)
- **`.stella`** - Explorable world with collision

To view the `.stella` file, install the VS Code extension from the GitHub repo.